In [1]:
using SeisNoise, SeisIO, Plots
using Dates 
using Plots
using SeisDvv
using CSV
using Statistics
using LinearAlgebra

d = load_corr("/data/wsd02/maleen_data/C_test/OO.HYSB1..BHZ.OO.HYSB1..BHE.jld2", "ZE")

CorrData with 389939 Corrs
      NAME: "OO.HYSB1..BHZ.OO.HYSB1..BHE"      
        ID: "2014-11-04"                       
       LOC: 0.0 N, 0.0 E, 0.0 m
      COMP: "ZE"                               
   ROTATED: false                              
 CORR_TYPE: "CC"                               
        FS: 40.0
      GAIN: 1.0
   FREQMIN: 0.01
   FREQMAX: 16.0
    CC_LEN: 1800.0
   CC_STEP: 450.0
  WHITENED: true                               
 TIME_NORM: ""                                 
      RESP: a0 1.0, f0 1.0, 1z, 1p
      MISC: 0 entries                          
     NOTES: 9 entries                          
      DIST: 0.0
       AZI: 0.0
       BAZ: 0.0
    MAXLAG: 100.0
         T: 2014-11-04T20:22:30                …
      CORR: 8001×389939 Matrix{Float32}        


In [11]:
using Dates

# Define the helper functions as previously described...

function stack_moving_window(corr_data, window_size_days, overlap_days)
    window_size = Day(window_size_days)
    overlap = Day(overlap_days)

    # Initialize a matrix for the stacked correlations and a vector for the start times
    stacked_correlations_matrix = Array{Float32}(undef, 8001, 0)
    stacked_start_times = Float64[]

    current_start_date = unixtimestamp_to_datetime(first(corr_data.t))

    while current_start_date <= unixtimestamp_to_datetime(last(corr_data.t))
        current_end_date = current_start_date + window_size - Day(1)
        indices = find_indices_in_window(corr_data.t, current_start_date, current_end_date)

        if length(indices) > 0
            window_data = deepcopy(corr_data)
            window_data.corr = corr_data.corr[:, indices]
            stacked_corr = stack(window_data, allstack=true).corr

            # Append the stacked correlation as a new column
            stacked_correlations_matrix = hcat(stacked_correlations_matrix, stacked_corr)

            # Append the start time of the window
            push!(stacked_start_times, datetime2unix(current_start_date))
        end

        current_start_date += overlap
    end

    # Create a new CorrData instance with the stacked data
    new_corr_data = deepcopy(corr_data)
    new_corr_data.corr = stacked_correlations_matrix
    new_corr_data.t = stacked_start_times

    return new_corr_data
end

# Test with a subset of your_corr_data for diagnostic purposes
# ...

new_corr_data = stack_moving_window(test_subset, 30, 15)


stack_moving_window (generic function with 1 method)

In [12]:
stacked_correlations_data = stack_moving_window(d, 30, 15)

LoadError: InterruptException:

In [45]:
window_size = Day(30)
overlap = Day(15)
corr_data=deepcopy(d)

# Initialize a matrix for the stacked correlations and a vector for the start times
stacked_correlations_matrix = Array{Float32}(undef, 8001, 0)
stacked_start_times = Float64[]

current_start_date = unixtimestamp_to_datetime(first(corr_data.t))

# Initialize a matrix for the stacked correlations and a vector for the start times
stacked_correlations_matrix = Array{Float32}(undef, 8001, 0)
stacked_start_times = Float64[]


Float64[]

In [47]:
for i in 1:20
    current_end_date = current_start_date + window_size - Day(1)
    indices = find_indices_in_window(corr_data.t, current_start_date, current_end_date)

    if length(indices) > 0
        window_data = deepcopy(corr_data)
        window_data.corr = corr_data.corr[:, indices]
        stacked_corr = stack(window_data, allstack=true).corr

        # Append the stacked correlation as a new column
        stacked_correlations_matrix = hcat(stacked_correlations_matrix, stacked_corr)

        # Append the start time of the window
        push!(stacked_start_times, datetime2unix(current_start_date))
    end

    new_corr_data = deepcopy(corr_data)
    new_corr_data.corr = stacked_correlations_matrix
    new_corr_data.t = stacked_start_times

    current_start_date += overlap
end

In [44]:
new_corr_data

CorrData with 6 Corrs
      NAME: "OO.HYSB1..BHZ.OO.HYSB1..BHE"      
        ID: "2014-11-04"                       
       LOC: 0.0 N, 0.0 E, 0.0 m
      COMP: "ZE"                               
   ROTATED: false                              
 CORR_TYPE: "CC"                               
        FS: 40.0
      GAIN: 1.0
   FREQMIN: 0.01
   FREQMAX: 16.0
    CC_LEN: 1800.0
   CC_STEP: 450.0
  WHITENED: true                               
 TIME_NORM: ""                                 
      RESP: a0 1.0, f0 1.0, 1z, 1p
      MISC: 0 entries                          
     NOTES: 9 entries                          
      DIST: 0.0
       AZI: 0.0
       BAZ: 0.0
    MAXLAG: 100.0
         T: 2014-11-04T20:22:30                …
      CORR: 8001×6 Matrix{Float32}             
